## 1 : langchainを使ってみた。

参考記事URL：
- https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_retrieval
- https://qiita.com/harukikaneko/items/b004048f8d1eca44cba9

In [1]:
!python --version

Python 3.11.9


In [2]:
import os
from dotenv import load_dotenv

# .envファイルを読み込む
load_dotenv()

#TODO: APIキーの登録が必要
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain_openai import OpenAIEmbeddings


openai_api_key = os.environ.get("OPENAI_API_KEY")

loader = PyPDFLoader("https://blog.freelance-jp.org/wp-content/uploads/2023/03/FreelanceSurvey2023.pdf")
pages = loader.load_and_split()
print(pages[0])

# 埋め込みを作成
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# def initialize_chroma(pages, embeddings):
#     # 遅延インポート
#     import chromadb.config
#     chroma_index = Chroma.from_documents(pages, embeddings)
#     return chroma_index

# chroma_index = initialize_chroma(pages, embeddings)
chroma_index = Chroma.from_documents(pages, embeddings)

# chroma_index = Chroma.from_documents(pages, OpenAIEmbeddings())
docs = chroma_index.similarity_search("「フリーランスのリモートワークの実態」について教えて。", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

page_content='第１章\nはじめに\n2' metadata={'source': 'https://blog.freelance-jp.org/wp-content/uploads/2023/03/FreelanceSurvey2023.pdf', 'page': 1}


/opt/anaconda3/envs/nnn_genai_webapp/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


44: ◼フリーランスのリモートワーク実態  (1/3)
45(n=850)Q.リモートワークの実施およびその影響に関して、それぞれ当てはまる選択肢を一つ選んでください。
（単一回答）
 ※リモートワークとは、情報通信技術 (ICT)を活用した、場所や時間にとらわれない柔軟な働き方のことを指します。
コロナ以前（ 2020年2月以前）と比べてリモートワークでのフリーランス業務が増えた
62.0%の回答者 が「リモートワーク業務が増えた」ことに同意。一方で、 11.3% が同意できないと回答し、「どちらとも
言えない」との回答も 26.7% 存在した。全体としてリモートワークの拡大傾向は見られるものの、就労形態に影響を及ぼ
さないフリーランスも一定数いることが見てとれる。
※小数点以下第二位四捨五入Copyright 2023 Freelance Association Japan.40.5%
21.5%26.7%4.1%7.2%
同意する割合は 62.0%
46: ◼フリーランスのリモートワーク実態 (3/3)
47Q.リモートワークの実施およびその影響に関して、それぞれ当てはまる選択肢を一つ選んでください。
（各項目単一回答）
 ※リモートワークとは、情報通信技術 (ICT)を活用した、場所や時間にとらわれない柔軟な働き方のことを指します。
3.3% 11.8% 41.3% 23.6% 20.0%リモートワーク普及により発注企業との関係構築が難しくなった
4.6% 10.1% 37.3% 26.1% 21.9%リモートワーク普及により仕事の負荷・ストレスが高まった
8.5% 25.3% 42.0% 15.3% 8.9%リモートワーク普及により仕事上の新しいつながりを築く機会が減った
リモートワーク普及の懸念点として、「発注企業との関係構築の困難さ」や 「仕事の負荷の増大」に同意する割合はそれぞ
れ2割に満たなかった 。一方で 、「仕事上の新しいつながりを築く機会が減った」ことに同意した割合は 33.8% であった。
既存クライアントとの関係や仕事の負荷に対するリモートワークの悪影響は限定的だが、 新たなつながりの創出においては
より懸念が大きいことが見てとれる
※小数点以下第二位四捨五入
(n=850)
Copyright 2023 Freelance Assoc

In [5]:
from langchain.text_splitter import CharacterTextSplitter
long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)
text_list = text_splitter.split_text(long_text)
print(text_list)
print(len(text_list))

document_list = text_splitter.create_documents([long_text])
print(document_list)
print(len(document_list))

368
['GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。\n\n自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。', 'これにより、人間と同じような文章を作成することが可能です。\n\nGPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。', 'GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。\n\nしかし、2021年9月までの情報しか持っていません。', 'このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。\n\nただし、GPT-4は完璧ではありません。\n\n時々、誤った情報や不適切な内容を生成することがあります。', '使用者は、その限界を理解し、\n\n適切な方法で利用することが重要です。']
5
[Document(page_content='GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。\n\n自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。'), Document(page_content='これにより、人間と同じような文章を作成することが可能です。\n\nGPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。'), Document(page_content='GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。\n\nしかし、2021年9月までの情報しか持っていません。'), Document(page_content='このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。\n\nただし、GPT-4は完璧ではありません。\n\n時々、誤った情報や不適切な内容を生成することがあります。'), Document(page_content='使用者は、その限界を理解し、\n\n適切な方法で利用することが重要です。')]
5


In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader

# from langchain.llms import OpenAI
# from langchain.chains import RetrievalQAWithSourcesChain

long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))
with open("./long_text.txt", "w") as f:
    f.write(long_text)
    f.close()

loader = TextLoader('./long_text.txt')

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)

index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, # Default
    embedding=OpenAIEmbeddings(), # Default
    text_splitter=text_splitter,
).from_loaders([loader])

query = "Q1. インターネット上の何のデータを使って、学習しているの？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

# answer_with_sources = index.query_with_sources(query)
# print(answer_with_sources)

query = "Q2. GPT4は第何世代のモデル？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

# answer_with_sources = index.query_with_sources(query)
# print(answer_with_sources)


368


Q1. インターネット上の何のデータを使って、学習しているの？

私はインターネット上の様々なデータを使って学習しています。例えば、ウェブサイトやブログ、オンラインコース、オンラインフォーラムなどから情報を収集し、それらを分析して学習しています。また、ソーシャルメディアやオンラインニュースなどからも情報を収集し、最新のトレンドやニュースを把握しています。さらに、オンラインのデータベースやオープンデータなどからも情報を収集し、それらを分析して学習しています。


Q2. GPT4は第何世代のモデル？
 第4世代目のモデルです。


In [11]:
long_text = """
１日中外に出ていれば、必ず何処かで一度は出会う警備業。

意識していなくても、私たちの安全は彼らに守られ、知らぬ間に支えられています。

スポットライトを浴びることは少ないかもしれませんが、そのニーズはコロナ禍でも減ることはなく、むしろ増加傾向を示しています。

警備業は、どんな時代でも、社会を支える存在であると言えるでしょう。

しかし、社会の縁の下の力持ちとも言える警備業ですが、そのうち93%の会社は警備員不足という難題を抱えています。

原因として、1.現職の高齢化、2.人材流入の低迷が挙げられます。

必要性は高くとも、アナログな働き方や、警備業へのイメージが、若い人材の足枷となり、流入を妨げてしまっています。

そこで、私たちは次の2軸に基づいて、みなさまへ新たな価値を提供します。
"""
print(len(long_text))
with open("./long_text.txt", "w") as f:
    f.write(long_text)
    f.close()

loader = TextLoader('./long_text.txt')

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)

index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, # Default
    embedding=OpenAIEmbeddings(), # Default
    text_splitter=text_splitter,
).from_loaders([loader])

query = "Q1. １日中外に出ていれば、必ず何処かで一度は何に出会いますか？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

# answer_with_sources = index.query_with_sources(query)
# print(answer_with_sources)

# query = "Q2. GPT4は第何世代のモデル？"
# print(f"\n\n{query}")
# answer = index.query(query)
# print(answer)

# answer_with_sources = index.query_with_sources(query)
# print(answer_with_sources)


352


Q1. １日中外に出ていれば、必ず何処かで一度は何に出会いますか？
 警備業


In [12]:
query = "Q2. 警備業とはどんな存在ですか？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)



Q2. 警備業とはどんな存在ですか？
 警備業は社会の縁の下で活躍する存在であり、私たちの安全を守り、社会を支える役割を担っています。しかし、高齢化や人材流入の低迷などの問題を抱えており、若い人材の流入を妨げるイメージがあります。それにもかかわらず、コロナ禍でも需要は減らず、むしろ増加傾向にあります。私たちは新たな価値を提供することで、警備業の重要性を再認識し、若い人材の流入を促すことを目指しています。
